# Initialization

In [1]:
# Display plots inline
%matplotlib inline

# Autoreload all package before excecuting a call
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import tailor
from tailor.clustering import *

In [3]:
data = tailor.load_data()

In [4]:
data.sample(10)

,article_id,season,brand,color,Abteilung,WHG,WUG,month,time_on_sale,original_price,sells_price,discount,markdown,article_count,stock_total,avq,revenue
142669,904900,Fall,Siofna,dunkelblau,Abteilung005,WHG022,WUG078,Jan,13,89.95,79.950000,0.000000,10.0,1.666667,274.0,29.075426,133.250000
79717,902742,Winter,Heimdall,hellgrau,Abteilung002,WHG012,WUG040,Aug,25,79.95,59.650000,0.300000,20.0,2.600000,531.0,62.561205,154.670000
52187,901818,Fall,Siofna,schwarz,Abteilung002,WHG015,WUG051,Apr,20,69.95,69.402500,0.547500,0.0,1.250000,288.0,58.420139,86.342500
223516,907655,Fall,Snotra,olivegrün,Abteilung002,WHG015,WUG055,Dec,13,109.95,88.437143,1.512857,20.0,3.571429,1099.0,20.772131,313.222857
169969,905821,Winter,Börr,schwarz,Abteilung002,WHG012,WUG040,Apr,12,19.95,19.865000,0.085000,0.0,85.000000,16260.0,14.037515,1689.185000
94090,903228,Spring,Gautr,schwarz / kombiniert,Abteilung005,WHG019,WUG063,Jul,12,49.95,49.740000,0.210000,0.0,3.666667,763.0,16.120577,182.100000
92685,903179,Spring,Tuisto,hellgrau,Abteilung002,WHG011,WUG038,Aug,24,39.95,13.960000,5.990000,20.0,3.000000,610.0,71.174863,41.880000
31771,901101,Fall,Nersihenae,orange,Abteilung007,WHG043,WUG141,Nov,7,39.95,37.950000,2.000000,0.0,2.000000,1155.0,5.930736,73.900000
100346,903448,Summer,Turstuahenae,schwarz / kombiniert,Abteilung007,WHG043,WUG145,Nov,20,89.95,80.955000,8.995000,0.0,1.000000,269.0,29.182156,80.955000
174714,905983,Spring,Alaisiagae,weiss / kombiniert,Abteilung007,WHG042,WUG138,Apr,2,64.95,64.878333,0.071667,0.0,7.000000,1134.0,5.496767,454.148333


# Code

In [7]:
%%time

split_results = cluster.multi_feature_split(data, distance.euclidean, 50)

CPU times: user 2min 54s, sys: 1.45 s, total: 2min 56s
Wall time: 3min 1s


In [8]:
split_results['Clusters'].index

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8'], dtype='object')

In [9]:
split_results['Clusters']['5'][0]['Features']

Abteilung    Abteilung005
WHG                WHG021
brand           Fimmilena
WUG                WUG073
season             Spring
dtype: object

In [10]:
# get all clusters that remained unsplit
leafs = list()

# iterate through all layers of the clustering
for layer in split_results['Clusters'].index:
    # add all layer leaves and remove leaf parents
    for add_cluster in split_results['Clusters'][layer]:
        # iterate until parent cluster is found then remove it
        for index, check_cluster in enumerate(leafs):
            if check_cluster['Name'] in add_cluster['Name']:
                # parent cluster found, remove it
                del leafs[index]
                # no more than one parent cluster, therefore exit second for loop
                break
        leafs.append(add_cluster)

In [11]:
for cluster in leafs:
    print(cluster['Name'] + ": " + str(cluster['DataFrame']['article_id'].nunique()))

0_4_16: 39
0_4_17: 19
0_4_18: 30
0_4_19: 7
0_4_20: 18
0_4_21: 42
0_4_22: 5
0_4_23: 19
0_4_24: 40
0_4_25: 30
0_4_26: 22
0_4_27: 23
0_4_28: 15
0_4_29: 6
0_4_30: 6
0_4_31: 14
0_4_32: 2
0_4_33: 3
0_6_6: 26
0_6_7: 19
0_6_8: 12
0_6_9: 6
0_6_10: 2
0_6_11: 2
0_6_12: 2
0_6_13: 5
0_1_1_3: 33
0_1_1_6: 40
0_1_1_7: 41
0_1_1_9: 44
0_1_1_11: 29
0_1_1_12: 17
0_1_1_13: 13
0_1_1_14: 27
0_1_1_15: 3
0_1_1_16: 29
0_1_1_17: 1
0_1_1_18: 2
0_1_1_19: 4
0_1_1_20: 2
0_1_1_21: 4
0_1_2_1: 28
0_1_2_4: 14
0_1_2_5: 21
0_1_2_6: 8
0_1_3_1: 44
0_1_3_2: 8
0_1_5_1: 14
0_1_5_2: 21
0_1_5_3: 3
0_1_5_4: 14
0_1_6_4: 9
0_1_6_5: 26
0_1_6_6: 14
0_1_6_7: 18
0_1_6_8: 8
0_1_6_9: 4
0_1_6_10: 1
0_1_6_11: 2
0_1_6_12: 1
0_1_6_13: 3
0_1_6_14: 1
0_1_6_15: 1
0_1_6_16: 4
0_1_6_17: 1
0_1_7_1: 4
0_1_7_2: 33
0_1_7_3: 15
0_2_2_2: 42
0_2_2_4: 19
0_2_3_6: 9
0_2_3_7: 37
0_2_3_8: 2
0_2_3_9: 11
0_2_3_10: 9
0_2_3_11: 5
0_3_3_2: 12
0_3_3_3: 1
0_3_3_4: 6
0_4_2_2: 44
0_4_3_3: 35
0_4_3_5: 16
0_4_3_6: 17
0_4_3_7: 33
0_4_3_8: 1
0_4_3_9: 10
0_4_3_10: 19
0_4

0_7_3_4_2: 34
0_7_3_4_3: 23
0_7_3_4_4: 37
0_7_3_4_5: 33
0_7_4_1_2: 1
0_7_4_1_3: 1
0_7_4_1_4: 3
0_7_4_1_5: 3
0_7_4_1_6: 9
0_7_4_1_7: 14
0_7_4_1_8: 10
0_7_4_1_9: 1
0_7_4_1_10: 1
0_7_4_1_11: 1
0_7_4_2_2: 10
0_7_4_2_3: 4
0_7_4_2_4: 37
0_7_4_2_5: 5
0_7_4_2_6: 3
0_7_4_2_7: 4
0_7_4_2_8: 2
0_7_4_2_9: 3
0_7_4_2_10: 6
0_7_4_2_11: 1
0_7_4_3_3: 50
0_7_4_4_2: 36
0_7_4_4_3: 23
0_7_4_4_4: 38
0_7_4_4_5: 35
0_7_5_1_2: 1
0_7_5_1_3: 1
0_7_5_1_4: 3
0_7_5_1_5: 3
0_7_5_1_6: 9
0_7_5_1_7: 14
0_7_5_1_8: 10
0_7_5_1_9: 1
0_7_5_1_10: 1
0_7_5_1_11: 1
0_7_5_2_2: 8
0_7_5_2_3: 13
0_7_5_2_4: 7
0_7_5_2_5: 50
0_7_5_2_6: 15
0_7_5_2_7: 3
0_7_5_2_8: 7
0_7_5_2_9: 2
0_7_5_2_10: 3
0_7_5_2_11: 1
0_7_5_4_2: 7
0_7_5_4_3: 7
0_7_5_4_4: 26
0_7_5_4_5: 36
0_7_5_4_6: 32
0_7_5_4_7: 13
0_7_5_4_8: 12
0_7_5_4_10: 3
0_7_5_4_11: 3
0_7_5_4_12: 4
0_7_5_4_13: 3
0_7_6_1_2: 2
0_7_6_1_3: 6
0_7_6_1_4: 9
0_7_6_1_5: 3
0_7_6_1_6: 6
0_7_6_1_7: 7
0_7_6_1_8: 4
0_7_6_1_9: 3
0_7_6_1_10: 7
0_7_6_1_11: 1
0_7_6_1_12: 2
0_7_6_1_13: 3
0_7_6_1_14: 1
0_7_6_1_15:

0_7_5_4_9_11: 3
0_7_5_4_9_12: 1
0_7_5_4_9_13: 2
0_7_5_4_9_14: 4
0_7_5_4_9_15: 4
0_7_5_4_9_16: 1
0_7_5_4_9_17: 1
0_7_5_4_9_18: 1
0_7_5_4_9_19: 1
0_7_5_4_9_20: 1
0_7_5_4_9_21: 1
0_7_5_4_9_22: 1
0_7_6_2_1_3: 1
0_7_6_2_1_4: 7
0_7_6_2_1_5: 15
0_7_6_2_1_6: 6
0_7_6_2_1_7: 3
0_7_6_2_1_8: 4
0_7_6_2_1_9: 2
0_7_6_2_1_10: 2
0_7_6_2_1_11: 1
0_7_6_2_1_12: 5
0_7_6_2_1_13: 7
0_7_6_2_1_14: 1
0_7_6_2_1_15: 4
0_7_6_2_1_16: 1
0_7_6_2_1_17: 1
0_7_6_2_1_18: 1
0_7_6_2_1_19: 1
0_7_6_2_1_20: 1
0_7_6_2_2_1: 6
0_7_6_2_2_2: 12
0_7_6_2_2_3: 31
0_7_6_2_2_4: 9
0_7_6_2_2_5: 7
0_7_6_2_2_6: 2
0_7_6_2_2_7: 1
0_7_6_2_2_8: 3
0_7_6_3_1_2: 2
0_7_6_3_1_3: 10
0_7_6_3_1_4: 23
0_7_6_3_1_5: 15
0_7_6_3_1_6: 10
0_7_6_3_1_7: 16
0_7_6_3_1_8: 2
0_7_6_3_1_9: 3
0_7_6_3_1_10: 3
0_7_6_3_1_11: 3
0_7_6_3_1_12: 2
0_7_6_3_1_13: 1
0_7_6_3_2_2: 16
0_7_6_3_2_3: 10
0_7_6_3_2_4: 12
0_7_6_3_2_5: 10
0_7_6_3_2_6: 6
0_7_6_3_2_7: 7
0_7_6_3_2_8: 1
0_7_6_3_2_9: 2
0_7_6_3_2_10: 3
0_7_6_3_3_1: 12
0_7_6_3_3_2: 25
0_7_6_3_3_3: 22
0_7_6_3_4_3: 3
0_7_6_3_4_4:

0_7_5_3_5_2_3: 10
0_7_5_3_5_2_4: 3
0_7_5_3_5_2_5: 4
0_7_5_3_5_2_6: 6
0_7_5_3_5_2_7: 2
0_7_8_3_1_1_2: 8
0_7_8_3_1_1_3: 15
0_7_8_3_1_1_4: 8
0_7_8_3_1_1_5: 3
0_7_8_3_1_1_6: 9
0_7_8_3_1_1_7: 6
0_7_8_3_1_1_8: 4
0_7_8_3_1_1_9: 1
0_7_8_3_1_1_10: 1
0_7_8_3_1_1_11: 1
0_7_8_3_1_1_12: 1
0_7_8_3_1_1_13: 1
0_7_8_3_1_1_14: 1
0_7_8_3_1_1_15: 1
0_7_8_3_1_1_16: 1
0_7_8_3_1_1_17: 1
0_7_8_3_1_3_1: 15
0_7_8_3_1_3_2: 2
0_7_8_3_1_3_3: 2
0_7_8_3_1_3_4: 10
0_7_8_3_1_3_5: 12
0_7_8_3_1_3_6: 7
0_7_8_3_1_3_7: 2
0_7_8_3_1_3_8: 1
0_7_8_3_1_4_1: 11
0_7_8_3_1_4_2: 17
0_7_8_3_1_4_3: 10
0_7_8_3_1_4_4: 3
0_7_8_3_1_4_5: 4
0_7_8_3_1_4_6: 6
0_7_8_3_1_4_7: 2


In [12]:
len(leafs)

2074